In [8]:
from selenium import webdriver
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import  WebDriverWait
import pandas as pd 
import time
import re
import os
import requests

In [ ]:
def scrape(path):

    # to avoid windows to close again and again we make use of headless
    options = ChromeOptions()
    options.headless=True
    # make use of chrome for scraping
    service = Service(ChromeDriverManager().install())
    # create a driver using chrome
    driver = Chrome(service=service, options=options)
    # run the driver through url
    driver.get(path)
    driver.maximize_window()
    
     # =====================================================================================
    # get the login icon and click on it 
# =====================================================================================
    # try:
    #         # get the login button
    #         login = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/span[text()="Login"]')))
    #         if login:
    #             driver.execute_script("arguments[0].scrollIntoView();", login)
    #             login.click()
    #         else:
    #             pass
    # except Exception as e:
    #     print(f"No login tab found and the error is {e}") 
    
    # #  get the username tab
    # try:
    #     provided_u_name = "fourbrotherstrading@icloud.com"
    #     user_name = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID,"logonIdentifier")))   
    #     user_name.send_keys(provided_u_name)
    # except Exception as e:
    #     print(f"No username tab found and the error is {e}")
    
    # # get password tab
    # try:
    #     provided_pass = "Muhssan7865"
    #     password = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "password")))
    #     driver.execute_script("arguments[0].scrollIntoView();", password)
    #     password.send_keys(provided_pass)
    # except Exception as e:
    #     print(f"No password tab found and the error is {e}") 
    
    # try:
    #     sign =  WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/button[text()= "Sign in"]')))
    #     if sign:
    #         sign.click()
    # except Exception as e:
    #     print(f"No sign in found {e}")
    
    
    
    
    # total cars
    # try:
    #     cars=  WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '(.//p[@class="MuiTypography-root MuiTablePagination-caption MuiTypography-body2 MuiTypography-colorInherit"])[2]')))
    #     if cars:
    #         tot_cars= int(cars.text.strip().split(" ")[2])
    #         print(f"found {tot_cars} cars ")
    # except Exception as e:
    #     print(f"No cars found {e}")
    
    # # get the check done
    # try:
    #     check = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/button[@id="rcc-confirm-button"]')))
    #     if check:
    #         driver.execute_script("arguments[0].scrollIntoView();", check)
    #         check.click()
        
    # except Exception as e:
    #     print(f"No check tab found and the error is {e}")
    
    try:
        page_main =WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="row gy-5"]'))) 
        if page_main:
            page_cars = WebDriverWait(page_main, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/a[text()="VIEW DETAILS"]')))
            if page_cars:
                total_cars = int(len(page_cars))
                print(f"Found {total_cars} cars")
    except Exception as e:
        print(e)
    
    
    # result
    results = []
    car_count = 0
    while car_count < total_cars: #total_cars
        
        try:
            page_cars= WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/a[text()="VIEW DETAILS"]')))
            
            for i in range(len(page_cars)): #total_cars
                if car_count >= total_cars:
                    break
                
                try:
                    page_cars = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/a[text()="VIEW DETAILS"]')))
                    driver.execute_script("arguments[0].scrollIntoView();", page_cars[i])
                    time.sleep(1)
                    page_cars[i].click()

                    details = {}
                    
                    # car name --> done
                    try:
                        title = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/h1[@id="carName"]'))).text.strip()
                        if title:
                            details['Title'] = title
                        else:
                            details['Title'] = "na"
                    except Exception as e:
                        print(f"No car title {e}")
                        
                    # lot --> done
                    try:
                        lot =WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/h2[@class="text-white bg-secondary mb-0"]'))).text.strip()
                        if lot:
                            details['Lot'] = lot.split(" ")[1]
                        else:
                            details['Lot'] = "na"
                    except Exception as e:
                        print("No lot num found")
                    
                    # images --> done
                    try:
                        imgs =  WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/li[@class="splide__slide carPreviewSlide"]')))
                        if imgs:
                            imgs_lst = [img.get_attribute("style").split(": ")[1].split('("')[1].split('")')[0] for img in imgs]
                            details['Images'] = ", ".join(imgs_lst)
                        else:
                            details['Images'] = "Na"
                    except Exception as e:
                        print("No car images")
                    
                    
                    # buy it now --> done
                    try:
                        buy = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/h4[@class="py-3 text-secondary"]'))).text.strip()
                        if buy:
                            details['Buy_it_now'] = buy.split(": ")[1]
                        else:
                            details['Buy_it_now'] = "na"
                    except Exception as e:
                        print("No buy information")        
                              
                    
                    # other information --> done
                    try:
                        info_card = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="row pt-5"]')))
                        if info_card:
                            body = WebDriverWait(info_card, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="row col-lg-9 col-12 p-0 m-0"]')))
                            if body:
                                trs = WebDriverWait(body, 5).until(EC.presence_of_all_elements_located((By.TAG_NAME, 'tr')))
                                if trs:
                                    for tr in trs:
                                        label = tr.find_element(By.TAG_NAME, 'th').text.strip()
                                        val = tr.find_element(By.TAG_NAME, 'td').text.strip()
                                        if label and val:
                                            details[label] = val
                                            
                            # get the check list--> done
                            check = WebDriverWait(info_card, 5).until(EC.presence_of_element_located((By.XPATH, './/a[text()="View Check Sheet"]'))).get_attribute("href")
                            if check:
                                details['Check_link'] = check
                            else:
                                details['Check_link'] = "na"
                        else:
                            print("No info card")
                    except Exception as e:
                        print("No info card found")
                    
                    # car description --> done
                    try:
                        desc =WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '(.//div[@class="mt-5"])[1]'))) 
                        if desc:
                            descrips = desc.find_elements(By.TAG_NAME, 'p')
                            if descrips:
                                desc_lst = [desc_val.text.strip() for desc_val in descrips]
                                details['Descriptions'] = desc_lst
                            else:
                                details['Descriptions'] = "na"
                    except Exception as e:
                        print("No car description found")
                                                
                    car_count+=1
                    results.append(details)
                    driver.back()

                except Exception as e:
                    print("No cars to click")
                    # break
        except Exception as e:
            print("No more cars")
            break

    df = pd.DataFrame.from_dict(results)
    df.to_csv("anglia.csv", index=False)
    # time.sleep(5)
    driver.quit()
    
path = "https://angliacarauctions.co.uk/auctions/2731-12-Nov-2025"
scrape(path)

Found 128 cars
No car images
No buy information
No info card found
No car images
No buy information
No info card found
No car images
No buy information
No info card found
No buy information
No info card found
No cars to click
No cars to click
No cars to click
No cars to click
No buy information
No cars to click
No cars to click
No cars to click
No cars to click
No cars to click
No cars to click
No cars to click
No cars to click
No cars to click
No cars to click
No cars to click
No cars to click
No cars to click
No cars to click
No cars to click
No cars to click
No cars to click
No cars to click
No cars to click
No cars to click
No cars to click
No cars to click
No cars to click
No cars to click
No cars to click
No cars to click
No cars to click
No cars to click
No cars to click
No cars to click
No cars to click
No cars to click
No cars to click
No cars to click
No cars to click
No cars to click
No cars to click
No cars to click
No cars to click
No cars to click
No cars to click
No cars

In [ ]:
df = pd.read_csv("anglia.csv")
df.columns

Index(['Title', 'Lot', 'Year', 'cc', 'Fuel', 'Mileage', 'Reg', 'CO2', 'Colour',
       'MOT', 'Type', 'Gearbox', 'Descriptions', 'Images', 'Former Keepers'],
      dtype='object')

## image downloader

### the image url is like --> /images/lots/2689/9330~1/0007_KTdCzX_lg.jpeg
### but to download an image using url we must have url like --> https://example.com/............
### therefore to make above url downloadable we need to add a part of url as a base url --> https://angliacarauctions.co.uk/..........
### the final url will be like that --> https://angliacarauctions.co.uk/images/lots/2689/9330~1/0007_KTdCzX_lg.jpeg

In [ ]:
from urllib.parse import urlparse, urljoin
from PIL import Image,ImageDraw,ImageFont
reg_img = df[["Reg", "Images"]]
def addWaterMark(image_path, text="Sourced from Angliacarauctions"):
    try:
        image = Image.open(image_path).convert("RGBA")
        txt_layer = Image.new("RGBA", image.size, (255, 255, 255, 0))
        draw = ImageDraw.Draw(txt_layer)

        # Smaller font for bottom-right
        font_size = max(10, image.width // 30)  # smaller than before
        try:
            font = ImageFont.truetype("arial.ttf", font_size)
        except:
            font = ImageFont.load_default()

        # text size
        bbox = draw.textbbox((0, 0), text, font=font)
        text_width = bbox[2] - bbox[0]
        text_height = bbox[3] - bbox[1]

        # bottom-right corner with margin
        margin = int(font_size * 0.6)
        x = image.width - text_width - margin
        y = image.height - text_height - margin

        # semi-transparent box
        box_padding = int(font_size * 0.5)
        draw.rectangle(
            [x - box_padding, y - box_padding, x + text_width + box_padding, y + text_height + box_padding],
            fill=(0, 0, 0, 180)
        )

        # draw text
        draw.text((x, y), text, font=font, fill=(255, 255, 255, 220))

        watermarked = Image.alpha_composite(image, txt_layer).convert("RGB")
        watermarked.save(image_path)
        print(f"✅ Watermark added to {image_path}")
    except Exception as e:
        print(f"❌ Failed to watermark {image_path}: {e}")

  



def download_images(data, main_folder="Images", base_url="https://angliacarauctions.co.uk"): 
    os.makedirs(main_folder, exist_ok=True)
    
    for index, row in data.iterrows():
        reg_no = row["Reg"] 
        image_urls = row["Images"]

        # Check if 'Images' column is empty or NaN
        if pd.isna(image_urls) or not isinstance(image_urls, str) or image_urls.strip() == "":
            print(f"Skipping {reg_no} (No image URLs)")
            continue

        image_urls = image_urls.split(", ")  # Split URLs by comma
        
        reg_folder = os.path.join(main_folder, reg_no)
        os.makedirs(reg_folder, exist_ok=True)
        
        for idx, url in enumerate(image_urls):
            url = url.strip()
            if not url.startswith(("http://", "https://")):
                url = urljoin(base_url, url) 
            
            parsed_url = urlparse(url)
            if not parsed_url.scheme or not parsed_url.netloc:
                print(f"Invalid URL skipped: {url}") 
                continue
            
            try:
                response = requests.get(url, stream=True) 
                response.raise_for_status()
                
                file_name = os.path.basename(parsed_url.path) or f"image_{idx + 1}.jpg"
                file_extension = file_name.split(".")[-1]
                
                if file_extension not in ["jpg", "jpeg", "png", "gif", "bmp", "webp"]:
                    file_name = f"image_{idx + 1}.jpg"
                
                full_file_name = os.path.join(reg_folder, f"{reg_no}_{idx + 1}_{file_name}")
                
                with open(full_file_name, 'wb') as f:
                    for chunk in response.iter_content(1024):
                        f.write(chunk)
                
                print(f"Downloaded: {full_file_name}")
                addWaterMark(full_file_name)
            except Exception as e:
                print(f"Failed to download {url} for {reg_no}: {e}")

# Call the function
download_images(reg_img)

Skipping PF08KXX (No image URLs)
Downloaded: Images\NL58EVR\NL58EVR_1_0007_IxZUra_lg.jpeg
✅ Watermark added to Images\NL58EVR\NL58EVR_1_0007_IxZUra_lg.jpeg
Downloaded: Images\NL58EVR\NL58EVR_2_0008_NGSLgi_lg.jpeg
✅ Watermark added to Images\NL58EVR\NL58EVR_2_0008_NGSLgi_lg.jpeg
Downloaded: Images\NL58EVR\NL58EVR_3_0009_MSoZhe_lg.jpeg
✅ Watermark added to Images\NL58EVR\NL58EVR_3_0009_MSoZhe_lg.jpeg
Downloaded: Images\NL58EVR\NL58EVR_4_0010_vvfPkn_lg.jpeg
✅ Watermark added to Images\NL58EVR\NL58EVR_4_0010_vvfPkn_lg.jpeg
Downloaded: Images\NL58EVR\NL58EVR_5_0011_xtdIIM_lg.jpeg
✅ Watermark added to Images\NL58EVR\NL58EVR_5_0011_xtdIIM_lg.jpeg
Downloaded: Images\NL58EVR\NL58EVR_6_0012_QrWXwQ_lg.jpeg
✅ Watermark added to Images\NL58EVR\NL58EVR_6_0012_QrWXwQ_lg.jpeg
Downloaded: Images\NL58EVR\NL58EVR_7_0013_cjjofG_lg.jpeg
✅ Watermark added to Images\NL58EVR\NL58EVR_7_0013_cjjofG_lg.jpeg
Downloaded: Images\NL58EVR\NL58EVR_8_0014_sGYCSZ_lg.jpeg
✅ Watermark added to Images\NL58EVR\NL58EVR_8_001